In [1]:
# Import necessary libraries
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [2]:
# Load the training data JSON file line by line 
train_data = []
with open('assignment1_training.json', 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Convert training data to DataFrame for easier processing
train_df = pd.DataFrame(train_data)

# Load the test data JSON file line by line 
test_data = []
with open('assignment1_test.json', 'r') as f:
    for line in f:
        test_data.append(json.loads(line))

# Convert test data to DataFrame for easier processing
test_df = pd.DataFrame(test_data)

In [3]:
train_df

,text,parent_comment,article_title,article_url,platform,platform_id,composite_toxic
0,"WTF, y'all never made MRE fart balloons in the...",None,Triangular UFO hovers over California military...,https://www.dailymail.co.uk/news/article-12112...,reddit,jlcm021,"[[False, 74], [True, 323], [False, 1028], [Fal..."
1,No apologies !! McCall has balls ! Ccp is not...,None,China sentences elderly US citizen to life in ...,https://www.cnn.com/2023/05/15/china/china-jai...,youtube,Ugws8gNW7eJyE9VHeM14AaABAg,"[[False, 216], [False, 197], [False, 1039], [F..."
2,What ever you need to tell yourself to sleep a...,I wonder how many undercover agents will be go...,Jan. 6 defendant who put foot on desk in Pelos...,https://www.cbsnews.com/news/richard-barnett-j...,youtube,UgxHlqwNcVssLHUr4yF4AaABAg.9q7kOunSlu-9q7lHH4he6S,"[[True, 192], [True, 193], [True, 260], [True,..."
3,@exZACKly @CBSNews Fuck off Nazi,@NCmylo @CBSNews Lol. Stop choosing to be an ...,19-year-old Missouri man arrested in U-Haul cr...,https://www.cbsnews.com/news/u-haul-crash-lafa...,twitter,1661025155047637000,"[[True, 92], [False, 218], [True, 69], [True, ..."
4,Texas is a republican sponsored killing ground...,None,At Least 8 Killed After Driver Plows Car Into ...,https://www.nytimes.com/2023/05/07/us/car-pede...,youtube,UgwpAfn9RIV0cHfhp4R4AaABAg,"[[False, 56], [True, 207], [False, 218], [Fals..."
...,...,...,...,...,...,...,...
3995,"Voluntary or not, won’t see my fat ass on any ...",Their ridership numbers are going to take a no...,Air New Zealand weighing passengers before flying,https://www.bbc.com/news/business-65765801,reddit,jmdgn83,"[[False, 81], [False, 184], [False, 248], [Fal..."
3996,So what?--*same party*. \n\nYou talk about hy...,May 2022. Whataboutisms. Haha. That was Kenny ...,Alberta declares state of emergency due to 'un...,https://edmonton.ctvnews.ca/alberta-declares-s...,reddit,jj7ixjw,"[[False, 46], [False, 89], [False, 112], [Fals..."
3997,@McKavitys Win stupid prizes! 😙,None,Bud Light parent company's stock downgraded by...,https://www.foxbusiness.com/markets/bud-light-...,twitter,1656631889661407233,"[[False, 95], [False, 45], [False, 323], [Fals..."
3998,@SIfill_ #DigitalSoldiers are afoot. An entire...,None,Atlanta police arrest 3 organizers behind bail...,https://www.pbs.org/newshour/politics/atlanta-...,twitter,1664307532570017795,"[[False, 182], [False, 196], [False, 33], [Fal..."


In [4]:
test_df

,text,parent_comment,article_title,article_url,platform,platform_id
0,Ukrainian Bullshit.,Russian Propaganda,Kremlin drone: Zelensky denies Ukraine attacke...,https://www.bbc.com/news/world-europe-65471904,youtube,UgxjV6HRpnD6FUmw8aV4AaABAg.9pH-CgX5yEH9pH7BMIfAz5
1,@LibDems No one likes you.\nYou denied democra...,None,"UK economy shrank 0.3% in March, ONS figures show",https://news.sky.com/story/uk-economy-shrank-0...,twitter,1657052099564150784
2,@EPurpera @BBCNews POS terrorist dictator Putr...,@BBCNews They should make peace talk.,Ukraine war: Kyiv hit by new massive Russian d...,https://www.bbc.com/news/world-65736730,twitter,1662672469205958656
3,@howardfineman @darkblue714 Bullshit. CNN set ...,None,Opinion | Why CNN's Trump town hall was always...,https://www.msnbc.com/opinion/msnbc-opinion/cn...,twitter,1656508255454019587
4,"The war will be won by who ""wins"" the race bet...",What is the pope gonna do? Pray and throw a co...,Zelenskyy to meet with Pope Francis at Vatican...,https://apnews.com/article/zelenskyy-italy-vis...,reddit,jk1pm1m
...,...,...,...,...,...,...
495,"Without accountability for killing, and displa...",None,Arab League poised to vote on restoring Syria ...,https://apnews.com/article/syria-arab-egypt-sa...,youtube,UgwitXUJxcPlN3AgqSZ4AaABAg
496,@ShlomoBen1 Don’t know anyone who does. Why?\n...,@GeneChurch1776 I'm asking.\n\nIs it what you ...,Uganda's president signs into law tough anti-g...,https://apnews.com/article/uganda-lgbtq-bill-s...,twitter,1663308218406252547
497,@CNN Why would CT CLAIM anything on his grand ...,None,ProPublica: GOP megadonor paid private school ...,https://www.cnn.com/2023/05/04/politics/claren...,twitter,1654293821251829760
498,"Who should we rely on, yours? or your friend W...",> So should I rely on China's assertion that a...,Biden sees shift in relations with China 'shor...,https://www.reuters.com/world/biden-sees-shift...,reddit,jl3p4n3


In [5]:
# Function to clean the text data
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text

# Apply cleaning to the 'text' column
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

# Extract the first element of composite_toxic as the target (True/False)
train_df['toxicity_label'] = train_df['composite_toxic'].apply(lambda x: x[0][0])

# Vectorize the cleaned text using TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X = tfidf.fit_transform(train_df['cleaned_text'])

# Target variable
y = train_df['toxicity_label']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1: Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Evaluate Logistic Regression
y_pred = log_reg.predict(X_val)
print("Logistic Regression Results:")
print(classification_report(y_val, y_pred))

# Model 2: Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Evaluate Random Forest
y_pred_rf = rf_clf.predict(X_val)
print("Random Forest Results:")
print(classification_report(y_val, y_pred_rf))

Logistic Regression Results:
              precision    recall  f1-score   support

       False       0.73      0.97      0.83       572
        True       0.60      0.11      0.18       228

    accuracy                           0.73       800
   macro avg       0.67      0.54      0.51       800
weighted avg       0.69      0.72      0.65       800

Random Forest Results:
              precision    recall  f1-score   support

       False       0.74      0.94      0.83       572
        True       0.53      0.16      0.24       228

    accuracy                           0.72       800
   macro avg       0.63      0.55      0.54       800
weighted avg       0.68      0.72      0.66       800



In [6]:
from sklearn.model_selection import GridSearchCV

# Logistic Regression Hyperparameter Tuning
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'solver': ['liblinear', 'saga', 'lbfgs'],  # Solver algorithms
    'max_iter': [200,200,1000], #increase iterations for convergence 
}

grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5, scoring='f1')
grid_search_lr.fit(X_train, y_train)

print(f"Best parameters for Logistic Regression: {grid_search_lr.best_params_}")
best_lr_model = grid_search_lr.best_estimator_

# Random Forest Hyperparameter Tuning
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [10, 20, 30],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at each leaf node
}

grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='f1')
grid_search_rf.fit(X_train, y_train)

print(f"Best parameters for Random Forest: {grid_search_rf.best_params_}")
best_rf_model = grid_search_rf.best_estimator_

# Evaluate the best models on the validation set
y_pred_best_lr = best_lr_model.predict(X_val)
print("Best Logistic Regression Results:")
print(classification_report(y_val, y_pred_best_lr))

y_pred_best_rf = best_rf_model.predict(X_val)
print("Best Random Forest Results:")
print(classification_report(y_val, y_pred_best_rf))

C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Clapton\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarni

Best parameters for Logistic Regression: {'C': 100, 'max_iter': 200, 'solver': 'saga'}
Best parameters for Random Forest: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Logistic Regression Results:
              precision    recall  f1-score   support

       False       0.75      0.80      0.78       572
        True       0.41      0.34      0.37       228

    accuracy                           0.67       800
   macro avg       0.58      0.57      0.57       800
weighted avg       0.65      0.67      0.66       800

Best Random Forest Results:
              precision    recall  f1-score   support

       False       0.72      1.00      0.84       572
        True       1.00      0.01      0.02       228

    accuracy                           0.72       800
   macro avg       0.86      0.50      0.43       800
weighted avg       0.80      0.72      0.60       800

